# Measure Pitch, HNR, Jitter, Shimmer, Formants, and Estimate VTL

### The extracted features
* voiceID
* duration
* meanF0Hz
* stdevF0Hz
* HNR
* localJitter
* localabsoluteJitter
* rapJitter
* ppq5Jitter
* ddpJitter
* localShimmer
* localdbShimmer
* apq3Shimmer
* apq5Shimmer
* apq11Shimmer
* ddaShimmer
* f1_mean
* f2_mean
* f3_mean
* f4_mean
* f1_median
* f2_median
* f3_median
* f4_median
* JitterPCA
* ShimmerPCA
* pF
* fdisp
* avgFormant
* mff
* fitch_vtl
* delta_f
* vtl_delta_f

## Import the external modules

In [141]:
#!/usr/bin/env python3
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics
import librosa
import noisereduce as nr
from pydub import AudioSegment


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## This function measures duration, pitch, HNR, jitter, and shimmer

In [142]:
# This is the function to measure source acoustics using default male parameters.

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

## This function measures formants at each glottal pulse

Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

Adapted from: DOI 10.17605/OSF.IO/K2BHS

In [143]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


## This function runs a 2-factor Principle Components Analysis (PCA) on Jitter and Shimmer

In [188]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

## Preprocessing to the audio file

In [153]:
def remove_silence(audio):
    unsilenced = []
    time_intervals = librosa.effects.split(audio, top_db=25, ref=np.max).tolist()
    for start, end in time_intervals:
        unsilenced += audio.tolist()[start:end+1]
    unsilenced = np.array(unsilenced)

    return unsilenced

def normalize(audio): 
    rms = np.sqrt(np.mean(audio**2))
    current_db = 20 * np.log10(rms)
    target_db = -20.0
    gain = target_db - current_db
    audio_normalized = audio * (10**(gain / 20))
    return audio_normalized

def load_mp3(path):
    audio_segment = AudioSegment.from_file(path, format="mp3")
    # Ensure the audio is mono
    audio_segment = audio_segment.set_channels(1)
    # Convert AudioSegment to raw PCM data
    samples = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)
    # Normalize the samples to the range [-1, 1]
    samples /= np.iinfo(audio_segment.array_type).max
    # Pass the samples and sampling rate to librosa
    sr = audio_segment.frame_rate
    return librosa.resample(samples, orig_sr=sr, target_sr=sr), sr

## This block of code runs the above functions on all of the '.wav' files in the /audio folder

In [ ]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []

failed_to_load = 0
# Go through all the wave files in the folder and measure all the acoustics
for file_path in glob.glob("../audio/*"):
    # audio, sr = librosa.load(file_path, sr=None, mono=True) if file_path[-3:] == 'wav' else load_mp3(file_path)
    audio, sr = None, None
    try:
        audio, sr = librosa.load(file_path, sr=None, mono=True) 
    except:
        failed_to_load += 1
        continue

    audio = remove_silence(audio)
    audio = normalize(audio)
    audio = nr.reduce_noise(y=audio, sr=sr)
    
    sound = parselmouth.Sound(audio, sampling_frequency=sr)
    
    (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
     localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(
        sound, 75, 300, "Hertz")
    
    (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(
        sound, file_path, 75, 300)
    
    file_list.append(file_path) # make an ID list
    duration_list.append(duration) # make duration list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr) #add HNR data
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
    
    # add the formant data
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)

print(f'Failed to load {failed_to_load} files')

## This block of code adds all of that data we just generated to a Pandas data frame

In [ ]:
# Add the data to Pandas
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_median_list, f2_median_list, f3_median_list, 
                                   f4_median_list]),
                                   columns=['voiceID', 'duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_median', 
                                            'f2_median', 'f3_median', 'f4_median'])

df.head()

,voiceID,duration,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,...,apq11Shimmer,ddaShimmer,f1_mean,f2_mean,f3_mean,f4_mean,f1_median,f2_median,f3_median,f4_median
0,../audio/common_voice_en_19721473.mp3,3.6589583333333335,197.90026381672538,36.01109473935635,12.566394872222943,0.014458740465676473,7.360629869052977e-05,0.0075539068514499695,0.007761952861042262,0.02266172055434991,...,0.09958144831112552,0.10849537850415555,509.0934252955819,1493.8265420381667,2126.9305474840135,3061.3890528656584,469.6031785696363,1255.784878523451,2036.851712024017,2891.06376444012
1,../audio/common_voice_en_19065733.mp3,4.512229166666667,205.21173891329184,38.502619374952125,11.249075130094742,0.01914458723801507,9.347859042476319e-05,0.008073747671123968,0.008430802357430236,0.024221243013371904,...,0.11165654114845838,0.14930871013051678,504.37270789501684,1503.906438722597,2171.456364503073,3316.5110349319334,458.1832425017985,1761.7745143516684,2335.2935408237513,3191.5271655583583
2,../audio/common_voice_en_19974678.mp3,3.2962083333333334,189.00044828098459,22.400259260405164,17.301504029675755,0.010254824771941933,5.4094815520572074e-05,0.004379576715892895,0.0038256444680614416,0.013138730147678685,...,0.07681303849196948,0.0754731591576304,466.5861016933732,1308.4915266620135,1957.8228954906958,2710.1777225671035,497.26529763951055,1177.6208086202637,1785.3618249679785,2683.245177303643
3,../audio/common_voice_en_19451222.mp3,6.1656875,234.13194926806614,21.26612756589558,13.544640215634992,0.015491709874319195,6.641067661439396e-05,0.007941498134769197,0.008324093286273765,0.023824494404307592,...,0.10437615800047509,0.14870247614449889,494.61264849700393,1486.337350412623,2330.6374132061364,3427.461947165166,483.38452529192307,1212.4375856204406,2413.3989184217253,3444.739589236262
4,../audio/common_voice_en_20080907.mp3,2.410791666666667,128.54488814422137,15.573467061943994,13.774125359926096,0.015148966656785926,0.00011816006290248828,0.006356180906902142,0.006914484333310453,0.019068542720706426,...,0.12933665103003575,0.1765774931168111,379.34327783993547,1168.638462882873,2108.2505918999286,2814.874791904987,353.09668803853066,1196.6543953346045,2020.0574182076666,2727.770043580041


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   voiceID              780 non-null    object
 1   duration             780 non-null    object
 2   meanF0Hz             780 non-null    object
 3   stdevF0Hz            780 non-null    object
 4   HNR                  780 non-null    object
 5   localJitter          780 non-null    object
 6   localabsoluteJitter  780 non-null    object
 7   rapJitter            780 non-null    object
 8   ppq5Jitter           780 non-null    object
 9   ddpJitter            780 non-null    object
 10  localShimmer         780 non-null    object
 11  localdbShimmer       780 non-null    object
 12  apq3Shimmer          780 non-null    object
 13  apq5Shimmer          780 non-null    object
 14  apq11Shimmer         780 non-null    object
 15  ddaShimmer           780 non-null    object
 16  f1_mean 

In [173]:
columns = df.columns
df[columns[1:]] = df[columns[1:]].astype('float64')

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   voiceID              780 non-null    object 
 1   duration             780 non-null    float64
 2   meanF0Hz             780 non-null    float64
 3   stdevF0Hz            780 non-null    float64
 4   HNR                  780 non-null    float64
 5   localJitter          780 non-null    float64
 6   localabsoluteJitter  780 non-null    float64
 7   rapJitter            780 non-null    float64
 8   ppq5Jitter           779 non-null    float64
 9   ddpJitter            780 non-null    float64
 10  localShimmer         780 non-null    float64
 11  localdbShimmer       780 non-null    float64
 12  apq3Shimmer          779 non-null    float64
 13  apq5Shimmer          779 non-null    float64
 14  apq11Shimmer         776 non-null    float64
 15  ddaShimmer           779 non-null    flo

In [186]:
df = df[~df.isna().any(axis=1)]

In [ ]:
pcaData = runPCA(df) # Run jitter and shimmer PCA
df = pd.concat([df, pcaData], axis=1) # Add PCA data
# reload the data so it's all numbers
df.to_csv("processed_results_2.csv", index=False)
# df = pd.read_csv('processed_results_2.csv', header=0)
df.sort_values('voiceID').head(20)

In [190]:
df.head()

,voiceID,duration,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,...,f1_mean,f2_mean,f3_mean,f4_mean,f1_median,f2_median,f3_median,f4_median,JitterPCA,ShimmerPCA
0,../audio/common_voice_en_19721473.mp3,3.658958,197.900264,36.011095,12.566395,0.014459,0.000074,0.007554,0.007762,0.022662,...,509.093425,1493.826542,2126.930547,3061.389053,469.603179,1255.784879,2036.851712,2891.063764,-2.093687,0.229653
1,../audio/common_voice_en_19065733.mp3,4.512229,205.211739,38.502619,11.249075,0.019145,0.000093,0.008074,0.008431,0.024221,...,504.372708,1503.906439,2171.456365,3316.511035,458.183243,1761.774514,2335.293541,3191.527166,-0.686959,-0.118547
2,../audio/common_voice_en_19974678.mp3,3.296208,189.000448,22.400259,17.301504,0.010255,0.000054,0.004380,0.003826,0.013139,...,466.586102,1308.491527,1957.822895,2710.177723,497.265298,1177.620809,1785.361825,2683.245177,-4.251508,-0.170616
3,../audio/common_voice_en_19451222.mp3,6.165687,234.131949,21.266128,13.544640,0.015492,0.000066,0.007941,0.008324,0.023824,...,494.612648,1486.337350,2330.637413,3427.461947,483.384525,1212.437586,2413.398918,3444.739589,-0.963093,-0.429421
4,../audio/common_voice_en_20080907.mp3,2.410792,128.544888,15.573467,13.774125,0.015149,0.000118,0.006356,0.006914,0.019069,...,379.343278,1168.638463,2108.250592,2814.874792,353.096688,1196.654395,2020.057418,2727.770044,-0.149377,-1.664218


## Next we calculate the vocal-tract length estimates

### Formant position
 Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

In [191]:
df['pF'] = (zscore(df.f1_median) + zscore(df.f2_median) + zscore(df.f3_median) + zscore(df.f4_median)) / 4

### Formant Dispersion
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [192]:
df['fdisp'] = (df['f4_median'] - df['f1_median']) / 3

### Fn (Average Formant)
Pisanski, K., & Rendall, D. (2011). The prioritization of voice fundamental frequency or formants in listeners’ assessments of speaker size, masculinity, and attractiveness. The Journal of the Acoustical Society of America, 129(4), 2201-2212.

In [193]:
df['avgFormant'] = (df['f1_median'] + df['f2_median'] + df['f3_median'] + df['f4_median']) / 4

### MFF 
Smith, D. R., & Patterson, R. D. (2005). The interaction of glottal-pulse rate and vocal-tract length in judgements of speaker size, sex, and age. The Journal of the Acoustical Society of America, 118(5), 3177-3186.

In [194]:
df['mff'] = (df['f1_median'] * df['f2_median'] * df['f3_median'] * df['f4_median']) ** 0.25

### Fitch VTL
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [196]:
# reload the data again
df.to_csv("processed_results_2.csv", index=False)
df = pd.read_csv('processed_results_2.csv', header=0)

df['fitch_vtl'] = ((1 * (35000 / (4 * df['f1_median']))) +
                   (3 * (35000 / (4 * df['f2_median']))) + 
                   (5 * (35000 / (4 * df['f3_median']))) + 
                   (7 * (35000 / (4 * df['f4_median'])))) / 4

### $\Delta$F 
Reby,D.,& McComb,K.(2003). Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [197]:
xysum = (0.5 * df['f1_median']) + (1.5 * df['f2_median']) + (2.5 * df['f3_median']) + (3.5 * df['f4_median'])
xsquaredsum = (0.5 ** 2) + (1.5 ** 2) + (2.5 ** 2) + (3.5 ** 2)
df['delta_f'] = xysum / xsquaredsum

### VTL($\Delta$F)
Reby,D.,&McComb,K.(2003).Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [198]:
df['vtl_delta_f'] = 35000 / (2 * df['delta_f'])

## Save the final data

In [199]:
# Write out the final dataframe
df.to_csv("processed_results_2.1.csv", index=False)

## Run this to tell you when it's done

In [200]:
print("finished")

finished
